In [1]:
from gurobipy import *
import gurobipy as gp

# type declaration
from typing import Dict, List


import pandas as pd
import numpy as np

# %load_ext nb_black
%load_ext lab_black

In [2]:
df_distances = pd.read_csv(
    "/home/angelo/Documents/Uni/Courses/Management Science/MM_Code_repo/data/distances.csv"
)
df_pairs = pd.read_csv(
    "/home/angelo/Documents/Uni/Courses/Management Science/MM_Code_repo/data/pairs.csv"
)

#### Data Prep

In [3]:
unique_c1 = list(df_distances.city1.unique())

# possible combinations
for c in unique_c1:
    d1 = [c, c, 0]
    df_distances = df_distances.append(
        pd.Series(d1, index=["city1", "city2", "distance"]), ignore_index=True
    )

/tmp/ipykernel_6410/3893934404.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_distances = df_distances.append(
/tmp/ipykernel_6410/3893934404.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_distances = df_distances.append(
/tmp/ipykernel_6410/3893934404.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_distances = df_distances.append(
/tmp/ipykernel_6410/3893934404.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_distances = df_distances.append(
/tmp/ipykernel_6410/3893934404.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead

In [4]:
df_pairs.loc[df_pairs.Dage == "-", "Dage"] = 99999999
df_pairs.Dage = df_pairs.Dage.astype(int)
df_pairs.loc[df_pairs.Rage == "-", "Rage"] = 99999999
df_pairs.Rage = df_pairs.Rage.astype(int)

df_pairs.loc[df_pairs.DBT == "-", "DBT"] = "NaN"
df_pairs.DBT = df_pairs.DBT.astype(str)
df_pairs.loc[df_pairs.RBT == "-", "RBT"] = "NaN"
df_pairs.RBT = df_pairs.RBT.astype(str)

In [5]:
# write the compatible bloodtypes
bloodtypes: Dict[str, int] = {
    # from A to everything
    ("A", "A"): 1,
    ("A", "B"): 0,
    ("A", "AB"): 1,
    ("A", "O"): 0,
    # from b to everything
    ("B", "A"): 0,
    ("B", "B"): 1,
    ("B", "AB"): 1,
    ("B", "O"): 0,
    # from AB to everything
    ("AB", "A"): 0,
    ("AB", "B"): 0,
    ("AB", "AB"): 1,
    ("AB", "O"): 0,
    # from O to everything
    ("O", "A"): 1,
    ("O", "B"): 1,
    ("O", "AB"): 1,
    ("O", "O"): 1,
    # missing blootypes
    ("NaN", "A"): 0,
    ("NaN", "B"): 0,
    ("NaN", "AB"): 0,
    ("NaN", "O"): 0,
    ("A", "NaN"): 0,
    ("B", "NaN"): 0,
    ("AB", "NaN"): 0,
    ("O", "NaN"): 0,
}

In [6]:
# change the order of the df to be more intuitive
RBT = df_pairs.loc[:, "RBT"]
DBT = df_pairs.loc[:, "DBT"]

# drop column
df_pairs.drop(columns=["RBT", "DBT"], axis=1, inplace=True)

# add them back
df_pairs["DBT"] = DBT
df_pairs["RBT"] = RBT

In [7]:
df_pairs["C"] = np.nan

In [8]:
for i, r in df_pairs.iterrows():
    if (
        (df_pairs.iloc[i].type == "pair")
        & (
            abs(df_pairs.iloc[i].Rage.astype(int) - df_pairs.iloc[i].Dage.astype(int))
            <= 10
        )
        & (bloodtypes[(df_pairs.iloc[i].DBT, df_pairs.iloc[i].RBT)] == 1)
    ) == True:
        df_pairs.loc[i, "C"] = 1
    else:
        df_pairs.loc[i, "C"] = 0

### create arcs

In [9]:
def pos_city_sublist(location_):
    """_summary_

    Args:
        location_ (_type_): _description_

    Returns:
        _type_: _description_
    """
    return list(
        df_distances[
            (df_distances.city1.astype(str) == location_)
            & (df_distances.distance <= 300)
        ].city2.values
    )


def blood_types_rbt(RBT_):
    """_summary_

    Args:
        RBT_ (_type_): _description_

    Returns:
        _type_: _description_
    """
    valid_bloodtypes_receiver_tmp = []
    for k, v in bloodtypes.items():
        if (k[1] == RBT_) & (v == 1):
            valid_bloodtypes_receiver_tmp.append(k[0])

    return valid_bloodtypes_receiver_tmp


def blood_types_dbt(DBT_):
    """_summary_

    Args:
        DBT_ (_type_): _description_

    Returns:
        _type_: _description_
    """
    valid_bloodtypes_donor_tmp = []
    for k, v in bloodtypes.items():
        if (k[0] == DBT_) & (v == 1):
            valid_bloodtypes_donor_tmp.append(k[1])

    return valid_bloodtypes_donor_tmp

In [10]:
arcs_: Dict[tuple, list] = {}

for i, r in df_pairs.iterrows():
    id_ = df_pairs.iloc[i].ID
    type_ = df_pairs.iloc[i].type
    Rage_ = df_pairs.iloc[i].Rage
    Dage_ = df_pairs.iloc[i].Dage
    location_ = df_pairs.iloc[i].Location
    compatible_ = df_pairs.iloc[i].C
    RBT_ = df_pairs.iloc[i].RBT
    DBT_ = df_pairs.iloc[i].DBT

    # pairs
    if type_ == "pair":

        # create a subset of cities that are valid
        possible_cities1 = pos_city_sublist(location_)

        # for receivers, create a donor vloodtype list that contains all the blood types that are compartible
        valid_bloodtypes_receiver = blood_types_rbt(RBT_)

        # same but inverted the donor blood types
        valid_bloodtypes_donor = blood_types_dbt(DBT_)

        arcs_[("pair_receiver", id_)] = list(
            df_pairs[
                (df_pairs.Location.isin(possible_cities1))
                & (abs(df_pairs.Dage - Rage_) <= 10)
                & (df_pairs.DBT.isin(valid_bloodtypes_receiver))
            ].ID.values
        )

        arcs_[("pair_donor", id_)] = list(
            df_pairs[
                (df_pairs.Location.isin(possible_cities1))
                & (abs(df_pairs.Rage - Dage_) <= 10)
                & (df_pairs.RBT.isin(valid_bloodtypes_donor))
            ].ID.values
        )

    elif type_ == "receiver":
        # grap the city and only select those rows in "city1" where  the df_distances.city.astye(str) == location_
        # create a subset of cities that are valid
        possible_cities1 = pos_city_sublist(location_)

        # for receivers, create a donor vloodtype list that contains all the blood types that are compartible
        valid_bloodtypes_receiver = blood_types_rbt(RBT_)

        arcs_[("receiver", id_)] = list(
            df_pairs[
                (df_pairs.Location.isin(possible_cities1))
                & (abs(df_pairs.Dage - Rage_) <= 10)
                & (df_pairs.DBT.isin(valid_bloodtypes_receiver))
            ].ID.values
        )

    elif type_ == "donor":
        # grap the city and only select those rows in "city1" where  the df_distances.city.astye(str) == location_
        # create a subset of cities that are valid
        possible_cities1 = pos_city_sublist(location_)

        # same but inverted the donor blood types
        valid_bloodtypes_donor = blood_types_dbt(DBT_)

        arcs_[("donor", id_)] = list(
            df_pairs[
                (df_pairs.Location.isin(possible_cities1))
                & (abs(df_pairs.Rage - Dage_) <= 10)
                & (df_pairs.RBT.isin(valid_bloodtypes_donor))
            ].ID.values
        )

    else:
        print("warning")
        break

In [11]:
arcs_l = []
for k, v in arcs_.items():
    for i in range(len(v)):
        if k[0] in ["donor", "pair_donor"]:
            arcs_l.append((k[1], v[i]))

In [12]:
len(arcs_l)

896

## Da model 

arcs_l

In [24]:
arcs_l

[(4, 30),
 (4, 47),
 (4, 101),
 (4, 138),
 (4, 183),
 (4, 187),
 (8, 8),
 (8, 22),
 (8, 27),
 (8, 107),
 (8, 168),
 (8, 178),
 (8, 194),
 (10, 11),
 (10, 15),
 (10, 33),
 (10, 35),
 (10, 37),
 (10, 45),
 (10, 86),
 (10, 97),
 (10, 160),
 (10, 185),
 (12, 6),
 (12, 120),
 (12, 174),
 (12, 198),
 (13, 134),
 (14, 53),
 (14, 75),
 (14, 125),
 (14, 164),
 (14, 191),
 (16, 137),
 (19, 2),
 (19, 182),
 (20, 4),
 (20, 33),
 (20, 44),
 (20, 45),
 (20, 49),
 (20, 59),
 (20, 60),
 (20, 71),
 (20, 79),
 (20, 93),
 (20, 94),
 (20, 115),
 (20, 126),
 (20, 127),
 (20, 133),
 (20, 146),
 (20, 165),
 (20, 167),
 (20, 169),
 (20, 190),
 (20, 192),
 (21, 2),
 (21, 182),
 (29, 29),
 (29, 65),
 (29, 89),
 (29, 99),
 (29, 152),
 (29, 196),
 (30, 133),
 (31, 1),
 (31, 20),
 (31, 24),
 (31, 30),
 (31, 42),
 (31, 46),
 (31, 47),
 (31, 48),
 (31, 69),
 (31, 76),
 (31, 83),
 (31, 96),
 (31, 101),
 (31, 103),
 (31, 104),
 (31, 111),
 (31, 122),
 (31, 124),
 (31, 128),
 (31, 129),
 (31, 138),
 (31, 145),
 (31, 15

In [21]:
# list of donors & pairs
d_p = list(df_pairs[df_pairs.type.isin(["pair", "pair"])].ID.values)

# list of receivers and pairs
r_p = list(df_pairs[df_pairs.type.isin(["receiver", "pair"])].ID.values)

In [23]:
bigM = len(arcs_l) + 100000

model = Model("Organ_trader")
x = model.addVars(d_p, r_p, vtype=GRB.BINARY, name="x")

model.addConstrs(gp.quicksum( ))
# for i in range(len(arcs_l)):
#     x[i] += 1

In [31]:
for i in arcs_l:
    print(i)
    break

(4, 30)


In [53]:
arcs_l

[(4, 30),
 (4, 47),
 (4, 101),
 (4, 138),
 (4, 183),
 (4, 187),
 (8, 8),
 (8, 22),
 (8, 27),
 (8, 107),
 (8, 168),
 (8, 178),
 (8, 194),
 (10, 11),
 (10, 15),
 (10, 33),
 (10, 35),
 (10, 37),
 (10, 45),
 (10, 86),
 (10, 97),
 (10, 160),
 (10, 185),
 (12, 6),
 (12, 120),
 (12, 174),
 (12, 198),
 (13, 134),
 (14, 53),
 (14, 75),
 (14, 125),
 (14, 164),
 (14, 191),
 (16, 137),
 (19, 2),
 (19, 182),
 (20, 4),
 (20, 33),
 (20, 44),
 (20, 45),
 (20, 49),
 (20, 59),
 (20, 60),
 (20, 71),
 (20, 79),
 (20, 93),
 (20, 94),
 (20, 115),
 (20, 126),
 (20, 127),
 (20, 133),
 (20, 146),
 (20, 165),
 (20, 167),
 (20, 169),
 (20, 190),
 (20, 192),
 (21, 2),
 (21, 182),
 (29, 29),
 (29, 65),
 (29, 89),
 (29, 99),
 (29, 152),
 (29, 196),
 (30, 133),
 (31, 1),
 (31, 20),
 (31, 24),
 (31, 30),
 (31, 42),
 (31, 46),
 (31, 47),
 (31, 48),
 (31, 69),
 (31, 76),
 (31, 83),
 (31, 96),
 (31, 101),
 (31, 103),
 (31, 104),
 (31, 111),
 (31, 122),
 (31, 124),
 (31, 128),
 (31, 129),
 (31, 138),
 (31, 145),
 (31, 15

In [55]:
arcs_l_dict = {}
for paring in arcs_l:
    arcs_l_dict[paring] = 1

In [91]:
model = gp.Model()

x = model.addVars(arcs_l, obj=1, vtype=GRB.BINARY, name="x")
model.ModelSense = gp.GRB.MAXIMIZE


# model.addConstrs(gp.quicksum(x[i,j] for i in arcs_l) <= 1 for i in arcs_l)

for pairing in arcs_l:
    lhs = x[pairing]
    model.addConstr(lhs <= 1)

for pairing in arcs_l:
    lhs = x[pairing]
    model.addConstr(lhs <= 1)

model.optimize()
model.printAttr("X")
print("Kidney Transplants", model.ObjVal)

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (linux64)

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 896 rows, 896 columns and 896 nonzeros
Model fingerprint: 0xc516a83f
Variable types: 0 continuous, 896 integer (896 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 896.0000000

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 896 

Optimal solution found (tolerance 1.00e-04)
Best objective 8.960000000000e+02, best bound 8.960000000000e+02, gap 0.0000%

    Variable            X 
-------------------------
     x[4,30]            1 
     x[4,47]            1 
    x[4,101]            1 
    x[4

In [90]:
model.write("problem.lp")
with open("problem.lp") as f:
    print(f.read())

\ LP format - for model browsing. Use MPS format to capture full model detail.
Maximize
  x[4,30] + x[4,47] + x[4,101] + x[4,138] + x[4,183] + x[4,187] + x[8,8]
   + x[8,22] + x[8,27] + x[8,107] + x[8,168] + x[8,178] + x[8,194]
   + x[10,11] + x[10,15] + x[10,33] + x[10,35] + x[10,37] + x[10,45]
   + x[10,86] + x[10,97] + x[10,160] + x[10,185] + x[12,6] + x[12,120]
   + x[12,174] + x[12,198] + x[13,134] + x[14,53] + x[14,75] + x[14,125]
   + x[14,164] + x[14,191] + x[16,137] + x[19,2] + x[19,182] + x[20,4]
   + x[20,33] + x[20,44] + x[20,45] + x[20,49] + x[20,59] + x[20,60]
   + x[20,71] + x[20,79] + x[20,93] + x[20,94] + x[20,115] + x[20,126]
   + x[20,127] + x[20,133] + x[20,146] + x[20,165] + x[20,167] + x[20,169]
   + x[20,190] + x[20,192] + x[21,2] + x[21,182] + x[29,29] + x[29,65]
   + x[29,89] + x[29,99] + x[29,152] + x[29,196] + x[30,133] + x[31,1]
   + x[31,20] + x[31,24] + x[31,30] + x[31,42] + x[31,46] + x[31,47]
   + x[31,48] + x[31,69] + x[31,76] + x[31,83] + x[31,96] + x[